In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import wordpunct_tokenize
import numpy as np

In [2]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tojune\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tojune\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df = pd.read_csv('cleaned_data_jt.csv')
clean_txt = df['clean_text_jt'].tolist()
txt_id = df['id'].tolist()
target = df['target'].tolist()

In [5]:
def tokenize_and_lemmatize(txt_data):
    #This function tokenizes and lemmatizes the raw text data
    #Input: a list of strings
    #Output: a list of token lists
    #tokenize
    token_mat = []
    for txt in clean_txt:
        token_list = wordpunct_tokenize(txt)
        token_mat.append(token_list)
        
    stop_words = set(stopwords.words('english'))
    #lemmatize    
    lem_token_mat = []
    lemmatizer = WordNetLemmatizer()
    for token_list in token_mat:
        #remove stop words
        lem_token_list = [w for w in token_list if not w in stop_words]
        #lemmatize nouns 
        lem_token_list = [lemmatizer.lemmatize(w) for w in lem_token_list]
        #lemmatize verbs
        lem_token_list = [lemmatizer.lemmatize(w, pos = 'v') for w in lem_token_list]
        #lemmatize adjs (omitted)
        
        lem_token_mat.append(lem_token_list)
    
    return lem_token_mat

In [6]:
lem_token_mat = tokenize_and_lemmatize(clean_txt)

In [7]:
def vocabulary_builder(token_mat):
    #This function builds a vocabulary based on a tokenized dataset
    #Input: a list of token lists
    #output: a dictionary key:token, value: id
    vocabulary = {}
    index = 0
    for token_list in token_mat:
        for token in token_list:
            if token not in vocabulary:
                vocabulary[token] = index
                index += 1
    return vocabulary

In [8]:
vocabulary = vocabulary_builder(lem_token_mat)

In [9]:
#download a pre-trained model using gensim's api
import gensim.downloader as api
embedding_model = api.load("glove-twitter-50")

C:\Users\Tojune\anaconda3\envs\py38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [98]:
def word_embedding(token_mat):
    #Input: a list of token lists
    #Output: a list of vector list
    vec_mat = []
    for token_list in token_mat:
        vect_list = []
        token_num = 0
        for token in token_list:
            #If the token is in the vocabulary (of pre-trained model), then vectorize it and count it. Otherwise, the token is omitted)
            if token in embedding_model:
                vect_list.append(embedding_model[token])
                token_num += 1
        #Store the list of vectors
        vec_mat.append(vect_list)

    return vec_mat

In [99]:
vec_mat = word_embedding(lem_token_mat)

In [93]:
def word_embedding_for_storage(token_mat):
    #Input: a list of token lists
    #Output: a list of concatenated vectors
    vec_mat = []
    for token_list in token_mat:
        vect_list = []
        token_num = 0
        for token in token_list:
            #If the token is in the vocabulary (of pre-trained model), then vectorize it and count it. Otherwise, the token is omitted)
            if token in embedding_model:
                vect_list.append(embedding_model[token])
                token_num += 1
        #The vectors (50 in length) of the tokens are concatenated to form a longer vector of length token_num*50
        vect_list = np.reshape(vect_list, (token_num*50,))
        #store the number of tokens at the end of the list
        np.append(vect_list, token_num)
        #Store the concatenated vector
        vec_mat.append(vect_list)
    
    print(len(vec_mat))
    return vec_mat

In [94]:
def save_vectors_to_csv(vec_mat):
    with open('vectorized_data_jt.csv', 'w') as file:
        for vec in vec_mat:
            file.writeline(vec)
    return

In [95]:
vec_mat = word_embedding(lem_token_mat)
save_vectors_to_csv(vec_mat)

7613


AttributeError: '_io.TextIOWrapper' object has no attribute 'writeline'

In [22]:
vec_list = vec_mat[0]
token_num = token_num_list[0]
print(len(vec_list))
print(token_num)

350
7


In [67]:
def load_vectors(filename):
    data = np.genfromtxt(filename)
    X = np.asarray([])
    for vec_list in data:
        token_num = vec_list[-1]
        vectors = np.reshape(vec_list[:-1], (token_num,50))
        np.append(X, vectors)
    
    return X

In [68]:
X= load_vectors('vectorized_data_jt.csv')

<ipython-input-67-8ee6001cf3ed>:2: UserWarning: genfromtxt: Empty input file: "vectorized_data_jt.csv"
  data = np.genfromtxt(filename)


In [70]:
print(len(X))

0
